In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from scipy import stats
import scipy
import statsmodels.api as sm
import csv 
import sys
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import datetime
from datetime import datetime

ModuleNotFoundError: No module named 'statsmodels'

# PCA analysis for termstructure of a bond curve

### Objectives
 - Learn how to perform dimensionality reduction on yield curve - PCA
 - Learn tools of risk management of the fixed-income instruments
 - Learn how to compute eigen values and eigen vectors and their properties 

In this tutorial we will learn about the classic example of how dimensionality reduction techniques - PCA in this case - can be used to calculate risks of the portfolio of bonds or any other fixed income instruments

In [ ]:
#We use 10 years of data of USD cuve which is composed from the reference bonds from BBG
dataset = pd.read_csv('Data_USDcurve.csv')
dataset["Date"] = pd.to_datetime(dataset["Date"], format='%Y%m%d', errors='ignore')
dataset = dataset.set_index("Date")
dataset.head()

In [ ]:
#Plot the time series of 3m yield 
%matplotlib inline
plt.plot(dataset['003M'].values, color = 'blue')
plt.legend((['Yield 3M'])) 

In [ ]:
#We have the following termstructure (month)
tenorstructure = [1,2,3,6,9,12,18,24,36,48,60,84,120,180,240,300,360]
len(tenorstructure)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

x = tenorstructure

fig, (ax1) = plt.subplots(1,figsize=(12,6))

sinc, = ax1.plot(tenorstructure, dataset.iloc[0],  
                 linestyle = '-', marker ='o', color='blue', lw='3')

#legend = plt.legend()

def animate(i):
    x = tenorstructure
    f = dataset.iloc[i]
    sinc.set_ydata(f)
    sinc.set_label('Date =' + str(dataset.index[i]))
    legend = plt.legend(loc='upper right')
    #legend.remove()
    #legend = plt.legend()
    #plt.set_label()
    
def init():
    ax1.set_xlim(-2.0,365.0)
    ax1.set_ylim(0.0,6.0)
    ax1.axhline(0, color = 'black', lw=1)
    plt.rcParams.update({'font.size':14})
    plt.grid()
    plt.xlabel('Tenor (month)')
    plt.ylabel('Yield (%)')
    plt.title('USD yield curve from 01.01.2007 to 30.12.2016')
    #plt.legend(loc = 'lower right', frameon = True)
    
    return sinc,

step = 1
steps = np.arange(0,len(dataset)-1,step)
ani = FuncAnimation(fig, animate, steps, init_func = init, interval = 200, blit = True)

In [ ]:
#Let us plot time series of several tenors
%matplotlib inline
dataset[-500:].plot(figsize=(20,10))

Assume we are trading the bonds and we have explosure to many of them. We want to calculate our risks and PnL. One way to do that is to calculate the moves of each tenor. However, due to large amount of tenors and curves this will be computationally expensive and risk structure becomes complex. For this reason we introduce PCA to this problem and will try to represent the whole curve movement by movement of principal components.

In [ ]:
# First let us take a look at the cross-tenor correlation matrix 
np.round(dataset.diff().dropna().corr(),2)

# How the correlation changes with change in tenor?

In [ ]:
#PCA
dataset_change = dataset.diff().dropna()
X = np.matrix(dataset_change) #Dataset in the matrix form
X_dm = X - np.mean(X,axis =0)#Normalise the data set to make it with zero mean acroos the tenors
Cov_X = np.cov(X_dm, rowvar = False)#Calculate the covariance matrix
eigen = np.linalg.eig(Cov_X) #Calculate the eigen values and eigen vectors
eig_values_X = np.matrix(eigen[0]) #Separate the eigen values
eig_vectors_X = np.matrix(eigen[1]) #Separate the eigen vectors
Y_dm = X_dm * eig_vectors_X #Calculate the principal components

In [ ]:
yields_trans = Y_dm.copy()

In [ ]:
eig_values_X #Let us take a look at eigen values

In [ ]:
#Plotting the furst three principal components
plt.figure(figsize =(14,8))
plt.plot(yields_trans[:,0:3])
plt.legend(['PC1','PC2','PC3'])

In [ ]:
#Let us now understand what the first principal components mean: calculate their correlations with 
pc_yields = dataset_change.copy()
pc_yields['Yield_PC1'] = yields_trans[:,0]
pc_yields['Yield_PC2'] = yields_trans[:,1]
pc_yields['Yield_PC3'] = yields_trans[:,2]

#Correlation
np.round(pc_yields.corr(),2)

In [ ]:
plt.figure(figsize =(14,8))
plt.plot(tenorstructure, pc_yields.corr()['Yield_PC1'][:17])
plt.plot(tenorstructure, pc_yields.corr()['Yield_PC2'][:17])
plt.plot(tenorstructure, pc_yields.corr()['Yield_PC3'][:17])
plt.title('Correlations of main PCs vs tenors')
plt.legend(['correlation of PC1','correlation of PC2','correlation of PC3'])

In [ ]:
#Let us understand how much movement in the curve we can explain by these the first three main components
var_explained =np.zeros(eig_values_X.shape[1])
var_explained_agg =np.zeros(eig_values_X.shape[1])

eig_values_X_mat = np.diagflat(np.array(eig_values_X))
eigen_values = eig_values_X_mat.diagonal() ##diagonal matrix of eigen values
eig_values_sum_all = np.sum(eigen_values) #all variance

for i in range(len(eigen_values)): 
    var_explained[i] = eigen_values[i] / eig_values_sum_all #calculate how much we can explain by individual PCs
    
    
    eig_val_sum = np.sum(eigen_values[0:i+1]) #calculate how much we can explain cumulatively
    var_explained_agg[i] = eig_val_sum / eig_values_sum_all 
    

print('')
print('\t \t \t PC1    PC2   PC3   PC4   PC5')
print('')
print('Variance explained:     ', np.round(var_explained[0:5],3))
print('Agg Variance explained: ', np.round(var_explained_agg[0:5],3))
print('')

In [ ]:
plt.plot(np.round(var_explained[0:5],3),'o',c='blue', alpha=0.4, label ='Variance explained')
plt.plot(np.round(var_explained_agg[0:5],3),'o',c='red', alpha=0.4, label = 'Agg Variance explained')
plt.title('Variance explained')
plt.xlabel('# PC')
plt.ylabel('Variance')
plt.legend()
plt.show()

## Consider demo case with only two tenors: 1M and 12M

In [ ]:
dataset.head()

In [ ]:
plt.scatter(dataset['001M'], dataset['012M'] ,c='blue', alpha=0.4)
plt.title('Scatter plot of 1M and 12M')
plt.xlabel('1M')
plt.ylabel('12M')
plt.show()

In [ ]:
X = np.matrix(pd.concat([dataset['001M'], dataset['012M']], axis=1)) #Dataset in the matrix form
X_dm = X - np.mean(X,axis =0)#Normalise the data set to make it with zero mean acroos the tenors
Cov_X = np.cov(X_dm, rowvar = False)#Calculate the covariance matrix
eigen = np.linalg.eig(Cov_X) #Calculate the eigen values and eigen vectors
eig_values_X = np.matrix(eigen[0]) #Separate the eigen values
eig_vectors_X = np.matrix(eigen[1]) #Separate the eigen vectors
Y_dm = X_dm * eig_vectors_X #Calculate the principal components

In [ ]:
eigen

In [ ]:
np.array(eig_vectors_X[:,0])*np.array(eig_vectors_X[:,1])

In [ ]:
np.array(X_dm[:,0])

In [ ]:
plt.scatter(np.array(X_dm[:,0]), np.array(X_dm[:,1]) ,c='blue', alpha=0.4)
plt.annotate('PC1', xy=(0.0, 0.0), xytext=(3*0.72626759,  3*0.68741209),
            arrowprops=dict(facecolor='red', shrink=0.05), ha='center', va='center')

#plt.annotate('PC2', xy=(0.0, 0.0), xytext=(-1*-0.68741209,  -1*0.72626759),
#            arrowprops=dict(facecolor='red', shrink=0.05), ha='center', va='center')
plt.title('Scatter plot of 1M and 12M')
plt.xlabel('1M')
plt.ylabel('12M')
plt.show()

In [ ]:
plt.scatter(np.array(X_dm[:,0]), np.array(X_dm[:,1]) ,c='blue', alpha=0.4)
plt.annotate('PC1', xy=(0.0, 0.0), xytext=(3*0.72626759,  3*0.68741209),
             arrowprops=dict(facecolor='red', shrink=0.05), ha='left', va='center')

plt.annotate('PC2', xy=(3*0.72626759 + -1*-0.68741209,  3*0.68741209 + -1*0.72626759), xytext=(3*0.72626759,  3*0.68741209),
                        arrowprops=dict(facecolor='red', shrink=0.05), ha='left', va='center')
plt.title('Scatter plot of 1M and 12M')
plt.xlabel('1M')
plt.ylabel('12M')
plt.show()